# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time
import json

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,papatowai,-46.5619,169.4708,18.40,47,43,6.22,NZ,1738535334
1,1,anadyr,64.7500,177.4833,0.28,100,100,17.88,RU,1738535335
2,2,avarua,-21.2078,-159.7750,30.03,74,34,5.14,CK,1738535336
3,3,longyearbyen,78.2186,15.6401,2.91,65,75,4.63,SJ,1738535337
4,4,alice springs,-23.7000,133.8833,29.75,35,2,2.57,AU,1738535339


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
city_map = city_data_df.hvplot.points(
    x="Lng",  
    y="Lat", 
    geo=True,  
    tiles="OSM", 
    frame_width=700,  
    frame_height=500,  
    size="Humidity", 
    scale=.3, 
    color="City", 
)

# Display the map
# YOUR CODE HERE
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# YOUR CODE HERE
max_temp = 40  # Maximum temperature
min_temp = -30  # Minimum temperature
max_humidity = 100 # Maximum humidity
max_cloudiness = 100  # Maximum cloudiness (%)
max_wind_speed = 11  # Maximum wind speed (m/s)

# Narrow down cities that fit criteria
ideal_cities_df = city_data_df[
    (city_data_df["Max Temp"] <= max_temp) &
    (city_data_df["Max Temp"] >= min_temp) &
    (city_data_df["Humidity"] <= max_humidity) &
    (city_data_df["Cloudiness"] == max_cloudiness) & 
    (city_data_df["Wind Speed"] <= max_wind_speed)
]

# Drop any rows with null values
# YOUR CODE HERE
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
# YOUR CODE HERE
ideal_cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,wangon,-7.5161,109.0539,24.34,92,100,1.39,ID,1738535343
9,9,tiksi,71.6872,128.8694,-25.95,95,100,9.33,RU,1738535344
10,10,college,64.8569,-147.8028,-9.40,83,100,0.00,US,1738535345
15,15,albany,-35.0169,117.8917,18.23,79,100,0.89,AU,1738535351
26,26,olonkinbyen,70.9221,-8.7187,-0.72,94,100,5.60,SJ,1738535364


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
# YOUR CODE HERE
hotel_df = ideal_cities_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
# YOUR CODE HERE
hotel_df["Hotel Name"] = ""

# Display sample data
# YOUR CODE HERE
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
8,wangon,ID,-7.5161,109.0539,92,
9,tiksi,RU,71.6872,128.8694,95,
10,college,US,64.8569,-147.8028,83,
15,albany,AU,-35.0169,117.8917,79,
26,olonkinbyen,SJ,70.9221,-8.7187,94,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set base URL
base_url = "https://api.geoapify.com/v2/places"

# Set parameters to search for a hotel
radius = 10000 # YOUR CODE HERE
params = {
    # YOUR CODE HERE
    "categories": "accommodation.hotel",  # Search for hotels
    "limit": 1,  # Get only the closest hotel
    "apiKey": geoapify_key  # Geoapify API key (from api_keys.py)
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    lat = row["Lat"]
    lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{lon},{lat},{radius}" # YOUR CODE HERE
    params["bias"] = f"proximity:{lon},{lat}" # YOUR CODE HERE

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params) # YOUR CODE HERE

    # Convert the API response to JSON format
    name_address = name_address.json() # YOUR CODE HERE

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # pause to avoid rate limiting
    time.sleep(1)
    
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
wangon - nearest hotel: No hotel found
tiksi - nearest hotel: Арктика
college - nearest hotel: BEST WESTERN PLUS Pioneer Park Inn
albany - nearest hotel: Best Western Albany Motel & Apartments
olonkinbyen - nearest hotel: No hotel found
motueka - nearest hotel: Hotel Motueka
tralee - nearest hotel: The Ashe Hotel
waitangi - nearest hotel: Hotel Chathams
chibougamau - nearest hotel: No hotel found
dompu - nearest hotel: No hotel found
tual - nearest hotel: Suita
stanley - nearest hotel: Shorty's Motel
thompson - nearest hotel: Thompson Inn
bethel - nearest hotel: Hampton Inn Danbury
pathardi - nearest hotel: No hotel found
mount isa - nearest hotel: Ibis Styles
corner brook - nearest hotel: No hotel found
udachny - nearest hotel: Вилюй
bizhbulyak - nearest hotel: No hotel found
whitehorse - nearest hotel: Town & Mountain Hotel
hawaiian paradise park - nearest hotel: No hotel found
benghazi - nearest hotel: ترجاب
tosontsengel - nearest hotel: Алтай
sirte - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
8,wangon,ID,-7.5161,109.0539,92,No hotel found
9,tiksi,RU,71.6872,128.8694,95,Арктика
10,college,US,64.8569,-147.8028,83,BEST WESTERN PLUS Pioneer Park Inn
15,albany,AU,-35.0169,117.8917,79,Best Western Albany Motel & Apartments
26,olonkinbyen,SJ,70.9221,-8.7187,94,No hotel found
...,...,...,...,...,...,...
664,mungwi,ZM,-10.1732,31.3694,96,No hotel found
667,yerofey pavlovich,RU,53.9667,121.9667,90,No hotel found
668,cananeia,BR,-25.0147,-47.9267,90,No hotel found
670,bokovskaya,RU,49.2290,41.8306,67,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
hotel_map = hotel_df.hvplot.points(
    x="Lng",  # Longitude
    y="Lat",  # Latitude
    geo=True,  # Geographic map
    tiles="EsriNatGeo",  # tile provider
    frame_width=700,  # Adjust as needed
    frame_height=500,  # Adjust as needed
    size="Humidity",  # Size by humidity
    scale=.5,  # Adjust scaling
    color="City",  # Color by city
    cmap="Set1",  # Or another colormap
    hover_cols=["City", "Country", "Humidity", "Hotel Name"]  # Include Hotel Name in hover information
).opts(tools=['hover']) # Enable hover tool


# Display the map
# YOUR CODE HERE
hotel_map


:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)